In [1]:
from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
current_dir ='/home/sanderroet/scripts/test_show/'

gro = app.GromacsGroFile(current_dir+'vis-md.gro')
top = app.GromacsTopFile(current_dir+'topol.top', periodicBoxVectors=gro.getPeriodicBoxVectors(),
                        includeDir='/home/sanderroet/top')




In [2]:
system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.1*unit.nanometer,
        constraints=app.HBonds, rigidWater=False, ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(310*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 310*unit.kelvin, 25))

print(integrator.getTemperature())
#platform = mm.Platform.getPlatformByName('CPU')

simulation =app.Simulation(top.topology, system, integrator)#,platform)
simulation.context.setPositions(gro.positions)

310.0 K


In [3]:
with open(current_dir+'eq_checkpoint.chk', 'rb') as f:
     simulation.context.loadCheckpoint(f.read())

In [4]:
simulation.reporters.append(app.StateDataReporter(stdout, 10, step=True, 
    time=True, potentialEnergy=False, kineticEnergy=False, totalEnergy=False, 
    temperature=True, volume=False, density=False, progress=True, 
    remainingTime=True, speed=True, totalSteps = 1000, separator='\t'))
simulation.reporters.append(app.PDBReporter(current_dir+'temp_test_prod.pdb', 1000))
simulation.reporters.append(app.DCDReporter(current_dir+'temp_test_prod.dcd', 10))
print('Running production')
simulation.step(1000)

Running production
#"Progress (%)"	"Step"	"Time (ps)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	10	0.6	305.963727393	0	--
2.0%	20	0.62	304.429211496	0.0834	33:51
3.0%	30	0.64	309.952782113	0.0725	38:32
4.0%	40	0.66	304.691071258	0.076	36:22
5.0%	50	0.68	305.496171554	0.073	37:29
6.0%	60	0.7	305.742487727	0.075	36:04
7.0%	70	0.72	306.169412591	0.0763	35:05
8.0%	80	0.74	307.365244214	0.0741	35:45
9.0%	90	0.76	308.125232772	0.0752	34:50
10.0%	100	0.78	307.276584539	0.0739	35:05
11.0%	110	0.8	305.394541346	0.0748	34:16
12.0%	120	0.82	305.914013598	0.0755	33:34
13.0%	130	0.84	304.342216338	0.0744	33:40
14.0%	140	0.86	305.065375567	0.0751	32:58
15.0%	150	0.88	304.323229085	0.0743	32:58
16.0%	160	0.9	303.434803205	0.0748	32:19
17.0%	170	0.92	306.415431399	0.0754	31:42
18.0%	180	0.94	303.93596626	0.0747	31:37
19.0%	190	0.96	304.485657803	0.0751	31:02
20.0%	200	0.98	305.298532685	0.0746	30:52
21.0%	210	1.0	304.092498591	0.0751	30:16
22.0%	220	1.02	306.150467836	0.0755	29:44
23.0%

In [5]:
with open(current_dir+'prod_checkpoint.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())